In [1]:
import json
import collections
from collections import defaultdict
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from collections import Counter
import math
from statistics import mean

import numpy as np
from numpy import dot
from numpy.linalg import norm

import scipy
from scipy.optimize import linear_sum_assignment
from scipy import stats
from scipy.stats import bootstrap

import pickle

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/anyaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [4]:
from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [5]:
with open('../sparse-1006.json') as f:
    sparse = json.load(f)
with open('../dense-74.json') as f:
    dense = json.load(f)
with open('../sparse-74.json') as f:
    sparse_74 = json.load(f)

In [6]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'font.size': 20
})

In [7]:
def clean(x):
    words = tokenize(x.lower())
    wl = [stemmer.stem(w) for w in words if w not in stop_words and (w.islower() or w.isalnum())]
    return wl

In [8]:
def save_pickle(savename, d):
    with open('./pickles/'+savename+'.pkl', 'wb') as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
def unpickle(savename):
    with open('./pickles/'+savename+'.pkl', 'rb') as handle:
        return pickle.load(handle)

In [10]:
sampled = [
  "page2-34.svg",
  "page9-46.svg",
  "page3-85.svg",
  "page7-107.svg",
  "page8-159.svg",
  "page6-203.svg",
  "page2-112.svg",
  "page1-116.svg",
  "page1-69.svg",
  "page8-234.svg",
  "page8-21.svg",
  "page5-75.svg",
  "page1-0.svg",
  "page5-59.svg",
  "page3-121.svg",
  "page6-164.svg",
  "page4-128.svg",
  "page5-136.svg",
  "page6-99.svg",
  "page7-14.svg",
  "page5-128.svg",
  "page9-27.svg",
  "page7-105.svg",
  "page6-162.svg",
  "page9-13.svg",
  "page1-128.svg",
  "page5-186.svg",
  "page3-72.svg",
  "page4-157.svg",
  "page3-182.svg",
  "page7-197.svg",
  "page7-180.svg",
  "page6-143.svg",
  "page7-81.svg",
  "page3-136.svg",
  "page5-64.svg",
  "page7-218.svg",
  "page3-128.svg",
  "page7-26.svg",
  "page6-78.svg",
  "page4-24.svg",
  "page5-153.svg",
  "page7-248.svg",
  "page5-244.svg",
  "page4-93.svg",
  "page5-28.svg",
  "page8-235.svg",
  "page5-200.svg",
  "page2-131.svg",
  "page8-183.svg",
  "page1-119.svg",
  "page5-232.svg",
  "page1-129.svg",
  "page4-162.svg",
  "page3-41.svg",
  "page6-180.svg",
  "page6-149.svg",
  "page1-105.svg",
  "page4-10.svg",
  "page5-178.svg",
  "page2-137.svg",
  "page3-35.svg",
  "page-A.svg",
  "page-B.svg",
  "page-C.svg",
  "page-D.svg",
  "page-E.svg",
  "page-F.svg",
  "page-G.svg",
  "page-H.svg",
  "page-I.svg",
  "page-J.svg",
  "page-K.svg",
  "page-L.svg",
]

# Segmentation Agreement

In [12]:
class Wrapper:
    def __init__(self, data):
        self.data=data
    def get_data(self):
        return self.data

In [13]:
'''
d: {1:str, 2:str, ..., 7:str}
returns s: e.g. [[1,2,5],[4],[3,6,7]]
'''
def make_sets(d):
    s=[]
    rev=defaultdict(list)
    for k,v in d.items():
        rev[v].append(k)
    for ann, ind_set in rev.items():
        s.append(ind_set)
    return s

def make_seg(data):
    file_to_seg ={}
    for file, ann_dicts in data.items():
        file_to_seg[file] = []
        for ann in ann_dicts:
            piece_dict = ann['piece']
            file_to_seg[file].append(Wrapper(make_sets(piece_dict)))
    return file_to_seg           

In [14]:
SPARSE_SEG = make_seg(sparse)

In [15]:
DENSE_SEG = make_seg(dense)

In [16]:
SPARSE74_SEG = make_seg(sparse_74)

In [17]:
# https://cstheory.stackexchange.com/questions/6569/edit-distance-between-two-partitions/6582#6582
def weight(s1,s2):
    return len(set(s1)&set(s2)) #weight is the # of pieces matching

'''
l1,l2: e.g. [[1,2,5],[4],[3,6,7]]
returns: len(l1)*len(l2) cost matrix matching elm from l1 to l2
'''
def cost_matrix(l1,l2):
    mat = np.zeros((len(l1),len(l2)))
    for i in range(len(l1)):
        for j in range(len(l2)):
            mat[i][j] = weight(l1[i],l2[j])
    return mat

'''
d1,d2: piece-annotation dictionaries
returns: number, higher value/cost == higher agreement (MAX number of pieces that do not change)
'''
def seg_agreement(d1,d2):
#     cost = cost_matrix(make_sets(d1),make_sets(d2))
    cost = cost_matrix(d1,d2)
    row_ind, col_ind = linear_sum_assignment(cost, True) # maximum weight matching
    return cost[row_ind, col_ind].sum()
            
# print(seg_agreement({1:'body',2:'body',3:'body',4:'body',5:'face',6:'side fin',7:'tail fin of whale'},{1:'road',2:'crosswalk',3:'crosswalk',4:'crosswalk',5:'grass',6:'grass',7:'road'}))

In [23]:
def calc_SA(input_data):
    '''
    [input_data]: a list of segmentations of a tangram, [[[1,2,3],[4],[5,6,7]], ...]
    '''
    mean_agr=0
#     print(input_data)
    l=len(input_data)
    for i in range(l-1):
        for j in range(i+1,l):
            mean_agr +=(seg_agreement(input_data[i].data,input_data[j].data)) 
    mean_agr /= l*(l-1)/2
#     print(mean_agr)
    RESULT.append(mean_agr)
    return mean_agr

In [24]:
def bootstrap_SA(file_to_seg, resample):
    k=0
    rerun=[]
    file_to_segagr = {}
    global RESULT
    RESULT=[]
    for file, seg in file_to_seg.items():      
        input_data=(seg,)
#         try:
        res=bootstrap(input_data, calc_SA, confidence_level=0.95, vectorized=False,\
                     n_resamples=resample, method='percentile')
#         except:
#             rerun.append(file)
#             continue
            
        print(k)
        k+=1
            
        # proceseds values
#         low = res.confidence_interval.low
#         high = res.confidence_interval.high
#         mean = (low+high)/2
#         err = high-mean
        
        try:
            assert len(RESULT)==resample
        except:
            print('***', len(RESULT), file)
        ci_l,ci_u=np.percentile(RESULT, [2.5, 97.5])
        m=np.mean(RESULT)
        file_to_segagr[file] = (ci_l, ci_u, m)
        RESULT=[]
        
#     print(rerun, len(rerun))
    
#     #these files are same distribution, so just use the normal ND
#     for file in rerun:
#         file_to_segagr[file] = (calc_SA(file_to_seg[file]), None)
#         print(file, file_to_segagr[file])

    return file_to_segagr

### sparse

In [25]:
#SPARSE 1006 (parsing old file sparse+12)
sparse_sa=bootstrap_SA(SPARSE_SEG, 1000)
save_pickle('CUSTOM_sparse_1000', sparse_sa)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [92]:
# sparse_SA = unpickle('sparse_file_to_segagr_1000')

### dense

In [26]:
dense_sa = bootstrap_SA(DENSE_SEG, 1000)
save_pickle('CUSTOM_dense_1000', dense_sa)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [94]:
# dense_SA = unpickle('dense_file_to_segagr_1000')